In [1]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats
import math
import scipy
from scipy.optimize import curve_fit
import seaborn as sns
from matplotlib import cm
import csv
import scipy.io as sio
import pandas as pd
from scipy.special import erf
import time
import torch
import os
import h5py
from scipy.special import k0

# plotting configuration
ratio = 1.7
figure_len, figure_width = 24*ratio, 18*ratio
title_font_size = 12*ratio
font_size_1, font_size_2 = 9*ratio, 9*ratio
legend_size = 18*ratio
line_width, tick_len = ratio, 4*ratio
marker_size = 5*ratio
plot_line_width = 2*ratio
hfont = {'fontname': 'Arial'}
marker_edge_width = 4
pal = sns.color_palette("deep")
sns.set(style='ticks')

In [2]:
# network setup
N = 30 # N * N units for each cell type
dx2deg2 = 6 * 6 # (degrees per lattice spacing) squared
deg = 6
center = int(((N+1)*N)/2)

l_stimulus = [0, 5, 15, 25, 35, 45, 55, 65, 75, 85]
n_stimulus = len(l_stimulus)
tau_E, tau_P, tau_S, tau_V = 0.02, 0.01, 0.01, 0.01 # time constants
alpha = 2 # power exponent for the input-output function

# simulation setup
T = 4 # simulation time
dt = 0.001

def calc_distance(i, j):
    """calculate the distance between two neurons,
        i and j are the indices of two neurons, i and j are from 0 to N*N-1 (31*30),
        x1 and y1 are the coordinate for the first neuron, x1 and y1 are from 0 to 30.
    """
    x1, y1 = divmod(i, N)
    x2, y2 = divmod(j, N)

    xd = np.minimum(np.abs(x1 - x2), N - np.abs(x1 - x2))
    yd = np.minimum(np.abs(y1 - y2), N - np.abs(y1 - y2))
    return np.sqrt(xd**2 + yd**2)

def calc_distance_matrix(N):
    x, y = np.arange(N), np.arange(N)
    x_grid, y_grid = np.meshgrid(x, y, indexing='ij')

    x1, x2 = np.reshape(x_grid, (-1, 1)), np.reshape(x_grid, (1, -1))
    y1, y2 = np.reshape(y_grid, (-1, 1)), np.reshape(y_grid, (1, -1))

    xd = np.minimum(np.abs(x1 - x2), N - np.abs(x1 - x2))
    yd = np.minimum(np.abs(y1 - y2), N - np.abs(y1 - y2))
    return np.sqrt(xd**2 + yd**2)

############# classes #############
# firing rates class for four different cell types: E, PV, SST, VIP
class FiringRates:
    def __init__(self):
        self.E = np.zeros(N * N)
        self.P = np.zeros(N * N)
        self.S = np.zeros(N * N)
        self.V = np.zeros(N * N)

# rate fields parameters for external input sources L4 and LM, from the data analysis file
class ParaRateFields:
    def __init__(self):
        # 4 parameters for classical stimuli with different stimulus size
        self.r1, self.r2 = np.zeros(n_stimulus), np.zeros(n_stimulus)
        self.sigma1, self.sigma2 = np.zeros(n_stimulus), np.zeros(n_stimulus)
        self.bs = 0 # baseline activity

class NetConnectivity:
    """ 1. connection strengths,
        2. width of the Gaussian connectivity for recurrent connections,
        3. biases.
    """
    def __init__(self):
        # peak of the Gaussian connectivity
        self.EE, self.EP, self.ES, self.EV, self.EL4, self.ELM, self.EX, self.EY, self.EZ = 0, 0, 0, 0, 0, 0, 0, 0, 0
        self.PE, self.PP, self.PS, self.PV, self.PL4, self.PLM, self.PX, self.PY, self.PZ = 0, 0, 0, 0, 0, 0, 0, 0, 0
        self.SE, self.SP, self.SS, self.SV, self.SL4, self.SLM, self.SX, self.SY, self.SZ = 0, 0, 0, 0, 0, 0, 0, 0, 0
        self.VE, self.VP, self.VS, self.VV, self.VL4, self.VLM, self.VX, self.VY, self.VZ = 0, 0, 0, 0, 0, 0, 0, 0, 0
        self.ELM1, self.ELM2 = 0, 0
        
        # width of the Gaussian connectivity, only the recurrent connections, values are consistent with those in mathematica
        self.sigma2EE2, self.sigma2EP2, self.sigma2ES2, self.sigma2EV2 = 2*7**2, 2*5**2, 2*7**2, 2*5**2
        self.sigma2PE2, self.sigma2PP2, self.sigma2PS2, self.sigma2PV2 = 2*5**2, 2*4**2, 2*5**2, 2*4**2
        self.sigma2SE2, self.sigma2SP2, self.sigma2SS2, self.sigma2SV2 = 2*7**2, 2*5**2, 2*7**2, 2*5**2
        self.sigma2VE2, self.sigma2VP2, self.sigma2VS2, self.sigma2VV2 = 2*5**2, 2*4**2, 2*7**2, 2*4**2
        
        # bias for four cell types, from the optimization file
        self.biasE, self.biasP, self.biasS, self.biasV = 0, 0, 0, 0

class ConnectivityMat:
    def __init__(self):
        self.EEm, self.EPm, self.ESm, self.EVm = np.zeros((N * N, N * N)), np.zeros((N * N, N * N)), np.zeros((N * N, N * N)), np.zeros((N * N, N * N))
        self.PEm, self.PPm, self.PSm, self.PVm = np.zeros((N * N, N * N)), np.zeros((N * N, N * N)), np.zeros((N * N, N * N)), np.zeros((N * N, N * N))
        self.SEm, self.SPm, self.SSm, self.SVm = np.zeros((N * N, N * N)), np.zeros((N * N, N * N)), np.zeros((N * N, N * N)), np.zeros((N * N, N * N))
        self.VEm, self.VPm, self.VSm, self.VVm = np.zeros((N * N, N * N)), np.zeros((N * N, N * N)), np.zeros((N * N, N * N)), np.zeros((N * N, N * N))

############# functions #############
def initialize_fr(fr):
    # set firing rates of different populations to their baseline values
    fr.E.fill(0.5996)
    fr.P.fill(1.35)
    fr.S.fill(0.6818)
    fr.V.fill(0.8437)

def load_optimized_networks_parameters(conn_file_path):
    data = torch.load(conn_file_path, map_location=torch.device('cpu'))
    optimized_network_tensor = data['gW']
    optimized_network = optimized_network_tensor.detach().cpu().numpy()
    optimized_network = optimized_network[:4, :]
    return optimized_network
    
def initialize_connectivity_bias(J, J_mat, opti_conn_mat, bias_vec, config):
    bs = np.array([0.5996, 1.35, 0.6818, 0.8437, 0.6092, 0.5996, 0.5996, 0.5996, 0.5996, 0.5996])
    opti_conn_mat[0, :] = opti_conn_mat[0, :]/(2 * np.sqrt(bs[0]))
    opti_conn_mat[1, :] = opti_conn_mat[1, :]/(2 * np.sqrt(bs[1]))
    opti_conn_mat[2, :] = opti_conn_mat[2, :]/(2 * np.sqrt(bs[2]))
    opti_conn_mat[3, :] = opti_conn_mat[3, :]/(2 * np.sqrt(bs[3]))

    J.EE, J.EP, J.ES, J.EV, J.EL4, J.ELM, J.EX, J.EY, J.EZ = opti_conn_mat[0, 0], -opti_conn_mat[0, 1], -opti_conn_mat[0, 2], -opti_conn_mat[0, 3], opti_conn_mat[0, 4], opti_conn_mat[0, 5], opti_conn_mat[0, 6], 0, 0 # opti_conn_mat[0, 8]
    J.PE, J.PP, J.PS, J.PV, J.PL4, J.PLM, J.PX, J.PY, J.PZ = opti_conn_mat[1, 0], -opti_conn_mat[1, 1], -opti_conn_mat[1, 2], -opti_conn_mat[1, 3], opti_conn_mat[1, 4], opti_conn_mat[1, 5], opti_conn_mat[1, 6], 0, 0 # opti_conn_mat[1, 8]
    J.SE, J.SP, J.SS, J.SV, J.SL4, J.SLM, J.SX, J.SY, J.SZ = opti_conn_mat[2, 0], -opti_conn_mat[2, 1], -opti_conn_mat[2, 2], -opti_conn_mat[2, 3], opti_conn_mat[2, 4], opti_conn_mat[2, 5], opti_conn_mat[2, 6], 0, 0 # opti_conn_mat[2, 8]
    J.VE, J.VP, J.VS, J.VV, J.VL4, J.VLM, J.VX, J.VY, J.VZ= opti_conn_mat[3, 0], -opti_conn_mat[3, 1], -opti_conn_mat[3, 2], -opti_conn_mat[3, 3], opti_conn_mat[3, 4], opti_conn_mat[3, 5], opti_conn_mat[3, 6], 0, 0 # opti_conn_mat[3, 8]
    J.ELM1, J.ELM2 = 0, 0
    J.biasE, J.biasP, J.biasS, J.biasV = bias_vec[0], bias_vec[1], bias_vec[2], bias_vec[3]
    
    # L4 related connectivity parameters
    sigma2L3L42 = 2.0 * 7.0**2
    J.EL4 = J.EL4 / (np.pi * sigma2L3L42)
    J.PL4 = J.PL4 / (np.pi * sigma2L3L42)
    J.SL4 = J.SL4 / (np.pi * sigma2L3L42)
    J.VL4 = J.VL4 / (np.pi * sigma2L3L42)
    
    # LM and X related connectivity parameters   
    p_ELM, q_ELM = 1, 0
    sigma2ELM2_1, sigma2ELM2_2 = 2.0 * 15.0**2, 2.0 * 15.0**2

    sigma2PLM2 = 2.0 * 15.0**2
    
    J.ELM1, J.ELM2 = 0, 0
    J.ELM1 = (J.ELM * p_ELM)/ (np.pi * sigma2ELM2_1)
    J.ELM2 = (J.ELM * q_ELM)/ (np.pi * sigma2ELM2_2)
    J.PLM = J.PLM / (np.pi * sigma2PLM2)
    J.SLM = J.SLM / (np.pi * sigma2PLM2)
    J.VLM = J.VLM / (np.pi * sigma2PLM2)
    
    sigma2SX2 = 2.0 * 15.0**2
    J.EX = J.EX / (np.pi * sigma2SX2)
    J.PX = J.PX / (np.pi * sigma2SX2)
    J.SX = J.SX / (np.pi * sigma2SX2)
    J.VX = J.VX / (np.pi * sigma2SX2)
    
    distances = calc_distance_matrix(N)
    d2_matrix = (distances ** 2) * dx2deg2  # Convert to squared degrees
    
    J_mat.EEm = J.EE * np.exp(-d2_matrix / J.sigma2EE2) / (np.pi * J.sigma2EE2)
    J_mat.EPm = J.EP * np.exp(-d2_matrix / J.sigma2EP2) / (np.pi * J.sigma2EP2)
    J_mat.ESm = J.ES * np.exp(-d2_matrix / J.sigma2ES2) / (np.pi * J.sigma2ES2)
    J_mat.EVm = J.EV * np.exp(-d2_matrix / J.sigma2EV2) / (np.pi * J.sigma2EV2)

    J_mat.PEm = J.PE * np.exp(-d2_matrix / J.sigma2PE2) / (np.pi * J.sigma2PE2)
    J_mat.PPm = J.PP * np.exp(-d2_matrix / J.sigma2PP2) / (np.pi * J.sigma2PP2)
    J_mat.PSm = J.PS * np.exp(-d2_matrix / J.sigma2PS2) / (np.pi * J.sigma2PS2)
    J_mat.PVm = J.PV * np.exp(-d2_matrix / J.sigma2PV2) / (np.pi * J.sigma2PV2)

    J_mat.SEm = J.SE * np.exp(-d2_matrix / J.sigma2SE2) / (np.pi * J.sigma2SE2)
    J_mat.SPm = J.SP * np.exp(-d2_matrix / J.sigma2SP2) / (np.pi * J.sigma2SP2)
    J_mat.SSm = J.SS * np.exp(-d2_matrix / J.sigma2SS2) / (np.pi * J.sigma2SS2)
    J_mat.SVm = J.SV * np.exp(-d2_matrix / J.sigma2SV2) / (np.pi * J.sigma2SV2)

    J_mat.VEm = J.VE * np.exp(-d2_matrix / J.sigma2VE2) / (np.pi * J.sigma2VE2)
    J_mat.VPm = J.VP * np.exp(-d2_matrix / J.sigma2VP2) / (np.pi * J.sigma2VP2)
    J_mat.VSm = J.VS * np.exp(-d2_matrix / J.sigma2VS2) / (np.pi * J.sigma2VS2)
    J_mat.VVm = J.VV * np.exp(-d2_matrix / J.sigma2VV2) / (np.pi * J.sigma2VV2)

def read_rf_parameters_X(para_X, stim_type):
    if stim_type == 0:         
        para_X.r1 = 1/8 * np.sqrt(np.asarray([0, 5, 15, 25, 35, 45, 55, 65, 75, 85]))
        para_X.r2 = np.asarray([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
        para_X.sigma1 = np.ones(10) * 15
        para_X.sigma2 = np.ones(10) * 15
        para_X.bs = 0
    else:
        para_X.r1 = np.asarray([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
        para_X.r2 = np.asarray([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
        para_X.sigma1 = np.ones(10) * 15
        para_X.sigma2 = np.ones(10) * 15
        para_X.bs = 0
    return para_X
        
def read_rf_parameters_Y(para_Y, stim_type):
    if stim_type == 0:
        para_Y.r1 = np.asarray([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
        para_Y.r2 = np.asarray([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
        para_Y.sigma1 = np.ones(10) * 15
        para_Y.sigma2 = np.ones(10) * 15
        para_Y.bs = 0
    else:
        para_Y.r1 = np.asarray([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
        para_Y.r2 = np.asarray([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
        para_Y.sigma1 = np.ones(10) * 15
        para_Y.sigma2 = np.ones(10) * 15
        para_Y.bs = 0
    return para_Y

def read_rf_parameters_Z(para_Z, stim_type):
    if stim_type == 0:
        para_Z.r1 = np.zeros(10)
        para_Z.r2 = np.zeros(10)
        para_Z.sigma1 = np.ones(10) * 15
        para_Z.sigma2 = np.ones(10) * 15
        para_Z.bs = 0
    else:
        para_Z.r1 = np.zeros(10)
        para_Z.r2 = np.zeros(10)
        para_Z.sigma1 = np.ones(10) * 15
        para_Z.sigma2 = np.ones(10) * 15
        para_Z.bs = 0

def define_L4_inputs(para_L4, H_L4, stim_type, stim_idx):
    
    # load L4 rate field, replace R_L4
    rate_field_inputs_L4 = sio.loadmat('data/rate_field_inputs_L4.mat')['rate_field_inputs']

    if stim_idx == 0:
        R_L4 = rate_field_inputs_L4[stim_type*9+stim_idx, :, :].reshape((900)) * 0 + 0.6092
    else:
        R_L4 = rate_field_inputs_L4[stim_type*9+stim_idx-1, :, :].reshape((900))
    
    sigma2L3L42 = 2.0 * 7.0**2

    distances = calc_distance_matrix(N)
    d2_matrix = (distances ** 2) * dx2deg2  # Convert to squared degrees
    
    H_matrix = np.exp(-d2_matrix / sigma2L3L42)
    H_L4 = dx2deg2 * H_matrix @ R_L4

    return H_L4
            
def define_LM_inputs(para_LM, H_LM, H_ELM1, H_ELM2, stim_type, stim_idx, config):
    # load LM rate field, replace R_LM
    rate_field_inputs_LM = sio.loadmat('data/rate_field_inputs_LM.mat')['rate_field_inputs']

    if stim_idx == 0:
        R_LM = rate_field_inputs_LM[stim_type*9+stim_idx, :, :].reshape((900)) * 0 + 0.5996
    else:
        R_LM = rate_field_inputs_LM[stim_type*9+stim_idx-1, :, :].reshape((900))
    
    sigma2ELM2_1, sigma2ELM2_2 = 2.0 * 15**2, 2.0 * 15**2
        
    sigma2PLM2 = 2.0 * 15.0**2

    distances = calc_distance_matrix(N)
    d2_matrix = (distances ** 2) * dx2deg2  # Convert to squared degrees
    
    H_LM_matrix = np.exp(-d2_matrix / sigma2PLM2)
    H_ELM1_matrix = np.exp(-d2_matrix / sigma2ELM2_1)
    H_ELM2_matrix = np.exp(-d2_matrix / sigma2ELM2_2)

    H_LM = dx2deg2 * H_LM_matrix @ R_LM
    H_ELM1 = dx2deg2 * H_ELM1_matrix @ R_LM
    H_ELM2 = dx2deg2 * H_ELM2_matrix @ R_LM

    return H_LM, H_ELM1, H_ELM2
        
def define_X_inputs(para_X, H_X, stim_type, stim_idx):
    sigma2X2_1 = 2.0 * para_X.sigma1[stim_idx]**2
    sigma2X2_2 = 2.0 * para_X.sigma2[stim_idx]**2
    
    distances_to_center_sq = np.array([calc_distance(i, center) for i in range(N * N)])**2
    R_X = para_X.r1[stim_idx] * np.exp(-(distances_to_center_sq*dx2deg2/sigma2X2_1)) - para_X.r2[stim_idx] * np.exp(-(distances_to_center_sq*dx2deg2/sigma2X2_2))

    sigma2SX2 = 2.0 * 15.0**2
    
    distances = calc_distance_matrix(N)
    d2_matrix = (distances ** 2) * dx2deg2  # Convert to squared degrees
    
    H_X_matrix = np.exp(-d2_matrix / sigma2SX2)

    H_X = dx2deg2 * H_X_matrix @ R_X
        
    return H_X

def define_Y_inputs(para_Y, H_Y, stim_type, stim_idx):
    sigma2Y2_1 = 2.0 * para_Y.sigma1[stim_idx]**2
    sigma2Y2_2 = 2.0 * para_Y.sigma2[stim_idx]**2
    
    distances_to_center_sq = np.array([calc_distance(i, center) for i in range(N * N)])**2
    R_Y = para_Y.r1[stim_idx] * np.exp(-(distances_to_center_sq*dx2deg2/sigma2Y2_1)) - para_Y.r2[stim_idx] * np.exp(-(distances_to_center_sq*dx2deg2/sigma2Y2_2))
    
    sigma2SY2 = 2.0 * 15.0**2
    
    distances = calc_distance_matrix(N)
    d2_matrix = (distances ** 2) * dx2deg2  # Convert to squared degrees
    
    H_Y_matrix = np.exp(-d2_matrix / sigma2SY2)

    H_Y = dx2deg2 * H_Y_matrix @ R_Y
        
    return H_Y

def define_Z_inputs(para_Z, J, H_Z, stim_type, stim_idx):
    sigma2Z12 = 2.0 * para_Z.sigma1[stim_idx]**2
    sigma2Z22 = 2.0 * para_Z.sigma2[stim_idx]**2
    
    distances_to_center_sq = np.array([calc_distance(i, center) for i in range(N * N)])**2
    R_Z = para_Z.r1[stim_idx] * np.exp(-(distances_to_center_sq*dx2deg2/sigma2Z12)) - para_Z.r2[stim_idx] * np.exp(-(distances_to_center_sq*dx2deg2/sigma2Z22))

    sigma2SZ2 = 2.0 * 15.0**2
    
    distances = calc_distance_matrix(N)
    d2_matrix = (distances ** 2) * dx2deg2  # Convert to squared degrees
    
    H_Z_matrix = np.exp(-d2_matrix / sigma2SZ2)

    H_Z = dx2deg2 * H_Z_matrix @ R_Z
        
    return H_Z

def calculate_current_L4(J, H_L4):
    TotInputE = J.EL4 * H_L4
    TotInputP = J.PL4 * H_L4
    TotInputS = J.SL4 * H_L4
    TotInputV = J.VL4 * H_L4
    return TotInputE, TotInputP, TotInputS, TotInputV

def calculate_current_LM(J, H_ELM1, H_ELM2, H_LM):
    TotInputE = J.ELM1 * H_ELM1 - J.ELM2 * H_ELM2
    TotInputP = J.PLM * H_LM
    TotInputS = J.SLM * H_LM
    TotInputV = J.VLM * H_LM
    return TotInputE, TotInputP, TotInputS, TotInputV

def calculate_current_X(J, H_X):
    TotInputE = J.EX * H_X
    TotInputP = J.PX * H_X
    TotInputS = J.SX * H_X
    TotInputV = J.VX * H_X
    return TotInputE, TotInputP, TotInputS, TotInputV

def calculate_current_Y(J, H_Y):
    TotInputE = J.EY * H_Y
    TotInputP = J.PY * H_Y
    TotInputS = J.SY * H_Y
    TotInputV = J.VY * H_Y
    return TotInputE, TotInputP, TotInputS, TotInputV

def calculate_current_Z(J, H_Z):
    TotInputE = J.EZ * H_Z
    TotInputP = J.PZ * H_Z
    TotInputS = J.SZ * H_Z
    TotInputV = J.VZ * H_Z
    return TotInputE, TotInputP, TotInputS, TotInputV

def calculate_recurrent_currents(fr_E, fr_PV, fr_SST, fr_VIP, J, J_mat):
    c_EE = dx2deg2 * np.matmul(J_mat.EEm, fr_E)
    c_PE = dx2deg2 * np.matmul(J_mat.PEm, fr_E)
    c_SE = dx2deg2 * np.matmul(J_mat.SEm, fr_E)
    c_VE = dx2deg2 * np.matmul(J_mat.VEm, fr_E)
    
    c_EP = dx2deg2 * np.matmul(J_mat.EPm, fr_PV)
    c_PP = dx2deg2 * np.matmul(J_mat.PPm, fr_PV)
    c_SP = dx2deg2 * np.matmul(J_mat.SPm, fr_PV)
    c_VP = dx2deg2 * np.matmul(J_mat.VPm, fr_PV)
    
    c_ES = dx2deg2 * np.matmul(J_mat.ESm, fr_SST)
    c_PS = dx2deg2 * np.matmul(J_mat.PSm, fr_SST)
    c_SS = dx2deg2 * np.matmul(J_mat.SSm, fr_SST)
    c_VS = dx2deg2 * np.matmul(J_mat.VSm, fr_SST)
    
    c_EV = dx2deg2 * np.matmul(J_mat.EVm, fr_VIP)
    c_PV = dx2deg2 * np.matmul(J_mat.PVm, fr_VIP)
    c_SV = dx2deg2 * np.matmul(J_mat.SVm, fr_VIP)
    c_VV = dx2deg2 * np.matmul(J_mat.VVm, fr_VIP)
    return c_EE, c_PE, c_SE, c_VE, c_EP, c_PP, c_SP, c_VP, c_ES, c_PS, c_SS, c_VS, c_EV, c_PV, c_SV, c_VV

def run_SSN(fr, J, J_mat, H_L4, H_ELM1, H_ELM2, H_LM, H_X, H_Y, H_Z): # Checked, fine
    RecInputE = np.matmul(J_mat.EEm, fr.E) + np.matmul(J_mat.EPm, fr.P) + np.matmul(J_mat.ESm, fr.S) + np.matmul(J_mat.EVm, fr.V)
    RecInputP = np.matmul(J_mat.PEm, fr.E) + np.matmul(J_mat.PPm, fr.P) + np.matmul(J_mat.PSm, fr.S) + np.matmul(J_mat.PVm, fr.V)
    RecInputS = np.matmul(J_mat.SEm, fr.E) + np.matmul(J_mat.SPm, fr.P) + np.matmul(J_mat.SSm, fr.S) + np.matmul(J_mat.SVm, fr.V)
    RecInputV = np.matmul(J_mat.VEm, fr.E) + np.matmul(J_mat.VPm, fr.P) + np.matmul(J_mat.VSm, fr.S) + np.matmul(J_mat.VVm, fr.V)
    
    TotInputE = J.EL4 * H_L4 + J.ELM1 * H_ELM1 - J.ELM2 * H_ELM2 + J.EX * H_X + J.EY * H_Y + J.EZ * H_Z + dx2deg2 * RecInputE + J.biasE
    TotInputP = J.PL4 * H_L4 + J.PLM * H_LM + J.PX * H_X + J.PY * H_Y + J.PZ * H_Z + dx2deg2 * RecInputP + J.biasP
    TotInputS = J.SL4 * H_L4 + J.SLM * H_LM + J.SX * H_X + J.SY * H_Y + J.SZ * H_Z + dx2deg2 * RecInputS + J.biasS
    TotInputV = J.VL4 * H_L4 + J.VLM * H_LM + J.VX * H_X + J.VY * H_Y + J.VZ * H_Z + dx2deg2 * RecInputV + J.biasV
    
    fr.E += dt * ((-fr.E + np.maximum(0.0, TotInputE) ** alpha) / tau_E)
    fr.P += dt * ((-fr.P + np.maximum(0.0, TotInputP) ** alpha) / tau_P)
    fr.S += dt * ((-fr.S + np.maximum(0.0, TotInputS) ** alpha) / tau_S)
    fr.V += dt * ((-fr.V + np.maximum(0.0, TotInputV) ** alpha) / tau_V)
    
    c_EE = dx2deg2 * np.matmul(J_mat.EEm, fr.E)
    c_PE = dx2deg2 * np.matmul(J_mat.PEm, fr.E)
    c_SE = dx2deg2 * np.matmul(J_mat.SEm, fr.E)
    c_VE = dx2deg2 * np.matmul(J_mat.VEm, fr.E)
    
    c_EP = dx2deg2 * np.matmul(J_mat.EPm, fr.P)
    c_PP = dx2deg2 * np.matmul(J_mat.PPm, fr.P)
    c_SP = dx2deg2 * np.matmul(J_mat.SPm, fr.P)
    c_VP = dx2deg2 * np.matmul(J_mat.VPm, fr.P)
    
    c_ES = dx2deg2 * np.matmul(J_mat.ESm, fr.S)
    c_PS = dx2deg2 * np.matmul(J_mat.PSm, fr.S)
    c_SS = dx2deg2 * np.matmul(J_mat.SSm, fr.S)
    c_VS = dx2deg2 * np.matmul(J_mat.VSm, fr.S)
    
    c_EV = dx2deg2 * np.matmul(J_mat.EVm, fr.V)
    c_PV = dx2deg2 * np.matmul(J_mat.PVm, fr.V)
    c_SV = dx2deg2 * np.matmul(J_mat.SVm, fr.V)
    c_VV = dx2deg2 * np.matmul(J_mat.VVm, fr.V)
    
    return c_EE, c_PE, c_SE, c_VE, c_EP, c_PP, c_SP, c_VP, c_ES, c_PS, c_SS, c_VS, c_EV, c_PV, c_SV, c_VV

In [3]:
N = 30
center = int(((N+1)*N)/2) # index of the center
distances_to_center = np.array([calc_distance(i, center) for i in range(N * N)]) # calculate the distance between every point in the 2D grid and the center
sorted_distance_indices = np.argsort(distances_to_center) # indices of the sorted distance in the original array
sorted_distances = distances_to_center[sorted_distance_indices] # the sorted distances
sorted_distances_degree = sorted_distances*6 # the sorted distances in degree

sorted_distances_degree = np.asarray(sorted_distances_degree)  # your input array
unique_sorted_distances_degree, unique_indices = np.unique(sorted_distances_degree, return_index=True)

fr, J, J_mat = FiringRates(), NetConnectivity(), ConnectivityMat()
para_L4, para_LM, para_X, para_Y, para_Z = (ParaRateFields() for _ in range(5))
H_L4, H_ELM1, H_ELM2, H_LM, H_X, H_Y, H_Z = (np.zeros(N * N) for _ in range(7))

n_stim_type = 1
shape = (N * N, len(l_stimulus) * n_stim_type)
pairs = [
    "EE", "PE", "SE", "VE",
    "EP", "PP", "SP", "VP",
    "ES", "PS", "SS", "VS",
    "EV", "PV", "SV", "VV",
    "EL4", "PL4", "SL4", "VL4",
    "ELM", "PLM", "SLM", "VLM",
    "EX", "PX", "SX", "VX",
    "EY", "PY", "SY", "VY",
    "EZ", "PZ", "SZ", "VZ",
]

l_stim_type_index = [0] # 0 for classical only
l_config = ['final_gaussian_classical']#, 'final_diff_of_gaussian_classical_1', 'final_diff_of_gaussian_classical_2']
l_title = ['gaussian classical']#, 'diff of gaussian classical 1', 'diff of gaussian classical 2']
l_b_check_simulation = [True] # always do a comparison of simulations first
l_baseline = sio.loadmat("data/baselines.mat")['baseline']

for b_check_simulation in l_b_check_simulation:

    for config_idx in range(len(l_config)):
        config = l_config[config_idx]
        print(config)
        folder_path = 'models/optimized_files_' + config + '/'
        model_names = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f)) and f != '.DS_Store']
        sorted_files = sorted(model_names, key=lambda name: float(name.replace('.pt', '')))
        sorted_numbers = sorted([float(f.replace('.pt', '')) for f in sorted_files])
        if b_check_simulation:
            n_selected_models = 1
        else:
            if len(sorted_files) > 10:
                n_selected_models = 10
            else:
                n_selected_models = len(sorted_files)

        bias_mat = sio.loadmat('data/bias_mat_' + config + '.mat')['bias_mat']

        for model_name_idx in range(n_selected_models):
            model_name = sorted_files[model_name_idx]
            print(model_name)
            conn_file_path = folder_path + model_name
            optimized_network = load_optimized_networks_parameters(conn_file_path)

            bias_vec = bias_mat[:, model_name_idx]
            initialize_connectivity_bias(J, J_mat, optimized_network, bias_vec, config)  # initialize connectivity and bias

            c_mats = {f"c_{p}_mat": np.full(shape, np.nan) for p in pairs}
            fr_E_mat = np.zeros((N * N, (len(l_stimulus)) * n_stim_type)) * np.nan
            fr_P_mat = np.zeros((N * N, (len(l_stimulus)) * n_stim_type)) * np.nan
            fr_S_mat = np.zeros((N * N, (len(l_stimulus)) * n_stim_type)) * np.nan
            fr_V_mat = np.zeros((N * N, (len(l_stimulus)) * n_stim_type)) * np.nan

            df = pd.read_csv('simulated_rate_fields/' + config + '_response_grid_' +  str(model_name) + '.csv')

            if b_check_simulation:
                for stim_type in l_stim_type_index:
                    if stim_type == 0:
                        s_stim_type = 'classical'
                    else:
                        s_stim_type = 'inverse'

                    read_rf_parameters_X(para_X, stim_type)

                    for stim_idx in range(len(l_stimulus)):
                        print("stimulus: " + str(l_stimulus[stim_idx]))

                        initialize_fr(fr)
                        H_L4 = define_L4_inputs(para_L4, H_L4, stim_type, stim_idx)
                        H_LM, H_ELM1, H_ELM2 = define_LM_inputs(para_LM, H_LM, H_ELM1, H_ELM2, stim_type, stim_idx, config)
                        H_X = define_X_inputs(para_X, H_X, stim_type, stim_idx)

                        for k in range(int(T / dt)):
                            if k == 0:
                                c_mats["c_EL4_mat"][:, stim_idx], c_mats["c_PL4_mat"][:, stim_idx], c_mats["c_SL4_mat"][:, stim_idx], c_mats["c_VL4_mat"][:, stim_idx] = calculate_current_L4(J, H_L4)
                                c_mats["c_ELM_mat"][:, stim_idx], c_mats["c_PLM_mat"][:, stim_idx], c_mats["c_SLM_mat"][:, stim_idx], c_mats["c_VLM_mat"][:, stim_idx] = calculate_current_LM(J, H_ELM1, H_ELM2, H_LM)
                                c_mats["c_EX_mat"][:, stim_idx], c_mats["c_PX_mat"][:, stim_idx], c_mats["c_SX_mat"][:, stim_idx], c_mats["c_VX_mat"][:, stim_idx] = calculate_current_X(J, H_X)
                                c_mats["c_EY_mat"][:, stim_idx], c_mats["c_PY_mat"][:, stim_idx], c_mats["c_SY_mat"][:, stim_idx], c_mats["c_VY_mat"][:, stim_idx] = calculate_current_Y(J, H_Y)
                                c_mats["c_EZ_mat"][:, stim_idx], c_mats["c_PZ_mat"][:, stim_idx], c_mats["c_SZ_mat"][:, stim_idx], c_mats["c_VZ_mat"][:, stim_idx] = calculate_current_Z(J, H_Z)

                            run_SSN(fr, J, J_mat, H_L4, H_ELM1, H_ELM2, H_LM, H_X, H_Y, H_Z)

                            if k == int(T / dt) - 1:
                                (c_mats["c_EE_mat"][:, stim_idx], c_mats["c_PE_mat"][:, stim_idx], c_mats["c_SE_mat"][:, stim_idx], c_mats["c_VE_mat"][:, stim_idx],
                                c_mats["c_EP_mat"][:, stim_idx], c_mats["c_PP_mat"][:, stim_idx], c_mats["c_SP_mat"][:, stim_idx], c_mats["c_VP_mat"][:, stim_idx],
                                c_mats["c_ES_mat"][:, stim_idx], c_mats["c_PS_mat"][:, stim_idx], c_mats["c_SS_mat"][:, stim_idx], c_mats["c_VS_mat"][:, stim_idx],
                                c_mats["c_EV_mat"][:, stim_idx], c_mats["c_PV_mat"][:, stim_idx], c_mats["c_SV_mat"][:, stim_idx], c_mats["c_VV_mat"][:, stim_idx]) = run_SSN(fr, J, J_mat, H_L4, H_ELM1, H_ELM2, H_LM, H_X, H_Y, H_Z)

                        fr_E_mat[:, stim_idx] = fr.E
                        fr_P_mat[:, stim_idx] = fr.P
                        fr_S_mat[:, stim_idx] = fr.S
                        fr_V_mat[:, stim_idx] = fr.V 

                sio.savemat('simulation_data_folder/fr_E_' + config + '_' + model_name + '.mat', mdict={'E': fr_E_mat})
                sio.savemat('simulation_data_folder/fr_P_' + config + '_' + model_name + '.mat', mdict={'P': fr_P_mat})
                sio.savemat('simulation_data_folder/fr_S_' + config + '_' + model_name + '.mat', mdict={'S': fr_S_mat})
                sio.savemat('simulation_data_folder/fr_V_' + config + '_' + model_name + '.mat', mdict={'V': fr_V_mat})
            else:
                for stim_type in l_stim_type_index:
                    if stim_type == 0:
                        s_stim_type = 'classical'
                    else:
                        s_stim_type = 'inverse'

                    read_rf_parameters_X(para_X, stim_type)

                    for stim_idx in range(len(l_stimulus)):
                        initialize_fr(fr)
                        fr_E, fr_PV, fr_SST, fr_VIP = np.zeros(N*N) * np.nan, np.zeros(N*N) * np.nan, np.zeros(N*N) * np.nan, np.zeros(N*N) * np.nan

                        # read the fire rates
                        fr_E_compact = df[(df['cell_type'] == 'PYR') & (df['size'] == l_stimulus[stim_idx]) & (df['inverse'] == (stim_type!=0))]['dr'].values
                        fr_PV_compact = df[(df['cell_type'] == 'PV') & (df['size'] == l_stimulus[stim_idx]) & (df['inverse'] == (stim_type!=0))]['dr'].values
                        fr_SST_compact = df[(df['cell_type'] == 'SST') & (df['size'] == l_stimulus[stim_idx]) & (df['inverse'] == (stim_type!=0))]['dr'].values
                        fr_VIP_compact = df[(df['cell_type'] == 'VIP') & (df['size'] == l_stimulus[stim_idx]) & (df['inverse'] == (stim_type!=0))]['dr'].values
                        
                        if stim_idx != 0:
                            for i in range(len(unique_sorted_distances_degree)):
                                idx = np.where((distances_to_center*6) == unique_sorted_distances_degree[i])[0]
                                fr_E[idx] = fr_E_compact[i] + l_baseline[0, 0]
                                fr_PV[idx] = fr_PV_compact[i] + l_baseline[0, 1]
                                fr_SST[idx] = fr_SST_compact[i] + l_baseline[0, 2]
                                fr_VIP[idx] = fr_VIP_compact[i] + l_baseline[0, 3]
                        else:
                            fr_E[:] = 0.5996
                            fr_PV[:] = 1.35
                            fr_SST[:] = 0.6818
                            fr_VIP[:] = 0.8437
                            
                        print("stimulus: " + str(l_stimulus[stim_idx]))

                        H_L4 = define_L4_inputs(para_L4, H_L4, stim_type, stim_idx)
                        H_LM, H_ELM1, H_ELM2 = define_LM_inputs(para_LM, H_LM, H_ELM1, H_ELM2, stim_type, stim_idx, config)
                        H_X = define_X_inputs(para_X, H_X, stim_type, stim_idx)

                        c_mats["c_EL4_mat"][:, stim_idx], c_mats["c_PL4_mat"][:, stim_idx], c_mats["c_SL4_mat"][:, stim_idx], c_mats["c_VL4_mat"][:, stim_idx] = calculate_current_L4(J, H_L4)
                        c_mats["c_ELM_mat"][:, stim_idx], c_mats["c_PLM_mat"][:, stim_idx], c_mats["c_SLM_mat"][:, stim_idx], c_mats["c_VLM_mat"][:, stim_idx] = calculate_current_LM(J, H_ELM1, H_ELM2, H_LM)
                        c_mats["c_EX_mat"][:, stim_idx], c_mats["c_PX_mat"][:, stim_idx], c_mats["c_SX_mat"][:, stim_idx], c_mats["c_VX_mat"][:, stim_idx] = calculate_current_X(J, H_X)
                        c_mats["c_EY_mat"][:, stim_idx], c_mats["c_PY_mat"][:, stim_idx], c_mats["c_SY_mat"][:, stim_idx], c_mats["c_VY_mat"][:, stim_idx] = calculate_current_Y(J, H_Y)
                        c_mats["c_EZ_mat"][:, stim_idx], c_mats["c_PZ_mat"][:, stim_idx], c_mats["c_SZ_mat"][:, stim_idx], c_mats["c_VZ_mat"][:, stim_idx] = calculate_current_Z(J, H_Z)

                        (c_mats["c_EE_mat"][:, stim_idx], c_mats["c_PE_mat"][:, stim_idx], c_mats["c_SE_mat"][:, stim_idx], c_mats["c_VE_mat"][:, stim_idx],
                        c_mats["c_EP_mat"][:, stim_idx], c_mats["c_PP_mat"][:, stim_idx], c_mats["c_SP_mat"][:, stim_idx], c_mats["c_VP_mat"][:, stim_idx],
                        c_mats["c_ES_mat"][:, stim_idx], c_mats["c_PS_mat"][:, stim_idx], c_mats["c_SS_mat"][:, stim_idx], c_mats["c_VS_mat"][:, stim_idx],
                        c_mats["c_EV_mat"][:, stim_idx], c_mats["c_PV_mat"][:, stim_idx], c_mats["c_SV_mat"][:, stim_idx], c_mats["c_VV_mat"][:, stim_idx]) = calculate_recurrent_currents(fr_E, fr_PV, fr_SST, fr_VIP, J, J_mat)
                        
            with h5py.File("simulation_data_folder/c_mats_" + config + "_" + model_name + ".h5", "w") as f:
                for key, arr in c_mats.items():
                    f.create_dataset(key, data=arr)

final_gaussian_classical
2.9814842343e-01.pt
stimulus: 0
stimulus: 5
stimulus: 15
stimulus: 25
stimulus: 35
stimulus: 45
stimulus: 55
stimulus: 65
stimulus: 75
stimulus: 85
done
